In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7d"
fs_method, fs_ratio = "cancelout", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9170 | AUPR: 0.6581 | Acc: 95.46%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9227 | AUPR: 0.6725 | Acc: 95.46%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9279 | AUPR: 0.6836 | Acc: 95.52%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9167 | AUPR: 0.6568 | Acc: 95.45%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9221 | AUPR: 0.6728 | Acc: 95.45%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9273 | AUPR: 0.6815 | Acc: 95.43%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9279 | AUPR: 0.6836 | Acc: 95.52%
Internal test: AUC: 0.9344 | AUPR: 0.7083 | Acc: 95.01%
External test: AUC: 0.9210 | AUPR: 0.6661 | Acc: 95.14%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9176 | AUPR: 0.6586 | Acc: 95.43%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9225 | AUPR: 0.6712 | Acc: 95.49%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9276 | AUPR: 0.6828 | Acc: 95.45%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9178 | AUPR: 0.6593 | Acc: 95.39%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9222 | AUPR: 0.6710 | Acc: 95.43%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9282 | AUPR: 0.6850 | Acc: 95.46%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9282 | AUPR: 0.6850 | Acc: 95.46%
Internal test: AUC: 0.9342 | AUPR: 0.7072 | Acc: 95.13%
External test: AUC: 0.9213 | AUPR: 0.6687 | Acc: 95.19%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9206 | AUPR: 0.6606 | Acc: 95.38%
model(C=10.0)
Valid:         AUC: 0.9205 | AUPR: 0.6602 | Acc: 95.39%
model(C=100.0)
Valid:         AUC: 0.9204 | AUPR: 0.6599 | Acc: 95.38%
best model(C=1.0)
Valid:         AUC: 0.9206 | AUPR: 0.6606 | Acc: 95.38%
Internal test: AUC: 0.9295 | AUPR: 0.6887 | Acc: 95.09%
External test: AUC: 0.9138 | AUPR: 0.6314 | Acc: 94.81%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9135 | AUPR: 0.6362 | Acc: 95.12%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9119 | AUPR: 0.6356 | Acc: 95.38%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9042 | AUPR: 0.5879 | Acc: 94.93%
best model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9135 | AUPR: 0.6362 | Acc: 95.12%
Internal test: AUC: 0.9202 | AUPR: 0.6602 | Acc: 94.94%
External test: AUC: 0.9100 | AUPR: 0.6289 | Acc: 94.75%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9149 | AUPR: 0.6541 | Acc: 89.31%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9156 | AUPR: 0.6581 | Acc: 89.66%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6586 | Acc: 89.71%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9138 | AUPR: 0.6581 | Acc: 89.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6612 | Acc: 90.29%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6610 | Acc: 90.02%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9138 | AUPR: 0.6581 | Acc: 89.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6612 | Acc: 90.29%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6610 | Acc: 90.02%
best model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6612 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9196 | AUPR: 0.6409 | Acc: 95.20%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9207 | AUPR: 0.6390 | Acc: 95.15%
model(kernel=linear, C=1)
Valid:         AUC: 0.9204 | AUPR: 0.6372 | Acc: 95.12%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9207 | AUPR: 0.6390 | Acc: 95.15%
Internal test: AUC: 0.9301 | AUPR: 0.6775 | Acc: 94.77%
External test: AUC: 0.9155 | AUPR: 0.6148 | Acc: 94.65%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9220 | AUPR: 0.6563 | Acc: 95.35%
model(criterion=squared_error)
Valid:         AUC: 0.9220 | AUPR: 0.6567 | Acc: 95.36%
best model(criterion=squared_error)
Valid:         AUC: 0.9220 | AUPR: 0.6567 | Acc: 95.36%
Internal test: AUC: 0.9280 | AUPR: 0.6838 | Acc: 95.11%
External test: AUC: 0.9207 | AUPR: 0.6632 | Acc: 95.15%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9122 | AUPR: 0.6203 | Acc: 95.22%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9138 | AUPR: 0.6239 | Acc: 95.19%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6259 | Acc: 95.19%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6259 | Acc: 95.19%
Internal test: AUC: 0.9219 | AUPR: 0.6787 | Acc: 94.91%
External test: AUC: 0.9144 | AUPR: 0.6208 | Acc: 94.67%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7632 | AUPR: 0.4395 | Acc: 94.91%
model(n_neighbors=5)
Valid:         AUC: 0.7920 | AUPR: 0.4879 | Acc: 95.12%
model(n_neighbors=7)
Valid:         AUC: 0.8094 | AUPR: 0.5173 | Acc: 95.19%
best model(n_neighbors=7)
Valid:         AUC: 0.8094 | AUPR: 0.5173 | Acc: 95.19%
Internal test: AUC: 0.8068 | AUPR: 0.5538 | Acc: 95.21%
External test: AUC: 0.8099 | AUPR: 0.5395 | Acc: 95.10%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8619 | AUPR: 0.5545 | Acc: 87.03%
model(criterion=log_loss)
Valid:         AUC: 0.8662 | AUPR: 0.5705 | Acc: 88.59%
model(criterion=entropy)
Valid:         AUC: 0.8662 | AUPR: 0.5705 | Acc: 88.59%
best model(criterion=log_loss)
Valid:         AUC: 0.8662 | AUPR: 0.5705 | Acc: 88.59%
Internal test: AUC: 0.8730 | AUPR: 0.6196 | Acc: 89.61%
External test: AUC: 0.8591 | AUPR: 0.5810 | Acc: 88.50%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8881 | AUPR: 0.4648 | Acc: 87.21%
best model()
Valid:         AUC: 0.8881 | AUPR: 0.4648 | Acc: 87.21%
Internal test: AUC: 0.9008 | AUPR: 0.5248 | Acc: 87.49%
External test: AUC: 0.8863 | AUPR: 0.4575 | Acc: 86.24%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)